In [1]:
import openai

In [2]:
client = openai.Client()

In [4]:
vector_store = client.vector_stores.create(
    name="Tutor de Apostilas"
)

In [19]:
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [20]:
file_batch.status

'completed'

In [21]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [22]:
assistant = client.beta.assistants.create(
    name="Tutor de Apostilas",
    instructions="Você é um tutor especializado em tecnologias emergentes.  \
    Você sabe responder perguntas sobre LLMs como OpenAI, Hugging Face, etc. \
    Caso você não saiba as respostas, seja sincero e fale que não sabe responder.",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    model="gpt-4-turbo-preview"
)

In [23]:
pergunta = "Conforme o Documento, o que é o Hugging Face?"

In [24]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta
)

/var/folders/vc/sckgbrmn3z341jpkwqmzrp8h0000gn/T/ipykernel_905/3685932091.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/vc/sckgbrmn3z341jpkwqmzrp8h0000gn/T/ipykernel_905/3685932091.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [25]:
# Executa a Trhead
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions= "Nome e usuário premium"
)

/var/folders/vc/sckgbrmn3z341jpkwqmzrp8h0000gn/T/ipykernel_905/1703593232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [26]:
# Aguarda a Thread rodar
import time

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

/var/folders/vc/sckgbrmn3z341jpkwqmzrp8h0000gn/T/ipykernel_905/3925348386.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [27]:
# Verificar a resposta

if run.status == "completed":
    mensagem = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagem)
else:
    print(f"Erro: {run.status}")    

/var/folders/vc/sckgbrmn3z341jpkwqmzrp8h0000gn/T/ipykernel_905/1142915008.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagem = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_jSsXO6TCwUnbsg6zYQOLf9AD', assistant_id='asst_wme3xquBDw2Ocq7htbje74mi', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=436, file_citation=FileCitation(file_id='file-3tJeojZkjPqGGLX2VQ9biD'), start_index=423, text='【4:0†LLM.pdf】', type='file_citation')], value='O Hugging Face é descrito como uma comunidade de código aberto que reúne centenas de milhares de modelos de contribuidores, os quais podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. Esta comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, embora ainda não tenha conseguido equiparar-se ao desempenho de sistemas como o GPT-4【4:0†LLM.pdf】.'), type='text')], created_at=1753162156, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_S5Lp6ozbPpOWyQVt491Ijc5v', sta

In [28]:
print(mensagem.data[0].content[0].text.value)

O Hugging Face é descrito como uma comunidade de código aberto que reúne centenas de milhares de modelos de contribuidores, os quais podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. Esta comunidade de código aberto está rapidamente alcançando o desempenho dos modelos proprietários, embora ainda não tenha conseguido equiparar-se ao desempenho de sistemas como o GPT-4【4:0†LLM.pdf】.
